In [1]:
import torch
import torchvision
import numpy as np
import cv2
import argparse
from PIL import Image
from torchvision.transforms import transforms as transforms
import os
from pathlib import Path
import matplotlib.pyplot as plt
import random
import os, json, cv2, numpy as np, matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import albumentations as A # Library for augmentations
import torchvision
import torch
import torch.nn as nn
import torchvision.utils as vutils

#Very hacky fix to make matplotlib work on my pc
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.

repo_dir = "c:/Users/James/git/3dGolfPoseDetection/"
save_dir = repo_dir + "/dev-nbs/saved-labels/"
downloaded_dir = repo_dir + "downloaded-data/"

: 

: 

In [ ]:
def label_img(img, club_coordinates, box_cords, line_width=4):
    grip, club = club_coordinates
    labelled = img.copy()
    cv2.circle(labelled, grip, 5, (255,0,255), -1)
    cv2.circle(labelled, club, 5, (255,0,255), -1)
    cv2.line(labelled, club, grip, (255, 0, 0), 2)
    cv2.rectangle(labelled, box_cords[:2], box_cords[2:], (0, 0, 255), line_width)
    return labelled

class GolfDataset(torch.utils.data.Dataset):
    def __init__(self, filenames, save_dir, annotate = False, resize_size = 640):
        self.save_dir = save_dir
        self.filenames = filenames # filenames of image locations
        self.resize_size = resize_size
        self.dtype = torch.float32
        self.img_transforms = transforms.ToTensor()
        self.annotate = annotate

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img_num = int(filename.split("_")[0])
        img = cv2.imread(self.save_dir+filename)
        assert img.shape[0] == img.shape[1], f"Image is not square: {img.shape} | {filename}"

        w = img.shape[0]
        r = self.resize_size / float(w)
        img = cv2.resize(img, (self.resize_size, self.resize_size), interpolation = cv2.INTER_AREA)
        all_cords = [int(x) for x in filename.split("_")[1].split("-")]
        all_cords = [int(x*r) for x in all_cords]
        grip, club, box_cords = all_cords[:2], all_cords[2:4], all_cords[4:]
        if self.annotate:
            img = label_img(img, [grip, club], box_cords)

        target = {}
        target["boxes"] = torch.Tensor([box_cords]).type(self.dtype)
        target["keypoints"] = torch.Tensor([grip, club]).type(self.dtype)
        
        img = Image.fromarray(img[:,:,::-1])
        img = self.img_transforms(img)      

        return img, target

batch_size = 32

filenames = os.listdir(save_dir)
dataset = GolfDataset(filenames, save_dir, annotate=True, resize_size = 640, annotate=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=False)

real_batch = next(iter(dataloader))
Image.fromarray((vutils.make_grid(real_batch[0], padding=2, normalize=True).permute(1,2,0).numpy()*255).astype(np.uint8))

In [ ]:
filenames = os.listdir(save_dir)
dataset = GolfDataset(filenames, save_dir, annotate=True, resize_size = 640)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=False)
real_batch = next(iter(dataloader))
Image.fromarray((vutils.make_grid(real_batch[0], padding=2, normalize=True).permute(1,2,0).numpy()*255).astype(np.uint8))

In [8]:
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(weights=torchvision.models.detection.KeypointRCNN_ResNet50_FPN_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False
out = nn.ConvTranspose2d(512, 2, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
model.roi_heads.keypoint_predictor.kps_score_lowres = out
model

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.